This notebook is for taking a closer look into the effects of new observations
(possibly fantasy observations) on the posterior mean and variance. It is intended more
 as a sanity check, to help with better understanding the behavior of lookahead PCS.

Let's start with SingleTaskGP. This will be representative for independent arms.

In [1]:
import torch
from botorch import fit_gpytorch_model
from botorch.models import SingleTaskGP
from botorch.models.transforms import Standardize
from botorch.sampling import SobolQMCNormalSampler
from botorch.test_functions import Branin
from gpytorch import ExactMarginalLogLikelihood


In [2]:
train_X = torch.linspace(0, 1, 20).view(-1, 2).repeat(3, 1)
test_func = Branin()
train_Y = Branin(noise_std=1.0)(train_X).view(-1, 1)
model = SingleTaskGP(
    train_X, train_Y, outcome_transform=Standardize(m=1)
)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll)

posterior = model.posterior(train_X[:10])
mean = posterior.mean
variance = posterior.variance

fm = model.fantasize(train_X[0].view(1, 1, 2), SobolQMCNormalSampler(4))
fm_posterior = fm.posterior(train_X[:10])
fm_mean = fm_posterior.mean
fm_variance = fm_posterior.variance

print(f"mean: {mean.view(-1)}")
print(f"variance: {variance.view(-1)}")

print(f"fm mean: {fm_mean.view(4, 10)}")
print(f"fm variance: {fm_variance.view(4, 10)}")

print(f"mean diff: {(mean - fm_mean).view(4, 10)}")
print(f"variance diff: {(variance - fm_variance).view(4, 10)}")


mean: tensor([55.1395, 51.9170, 48.6181, 45.5666, 42.6158, 39.6841, 36.7813, 33.7308,
        30.9662, 28.7889], grad_fn=<ViewBackward>)
variance: tensor([0.3276, 0.2162, 0.2026, 0.2000, 0.1997, 0.1997, 0.2000, 0.2027, 0.2161,
        0.3276], grad_fn=<ViewBackward>)
fm mean: tensor([[55.2837, 51.9466, 48.6078, 45.5620, 42.6173, 39.6851, 36.7811, 33.7306,
         30.9663, 28.7890],
        [55.3597, 51.9623, 48.6023, 45.5596, 42.6181, 39.6857, 36.7810, 33.7304,
         30.9663, 28.7890],
        [54.9628, 51.8808, 48.6307, 45.5722, 42.6139, 39.6827, 36.7814, 33.7310,
         30.9663, 28.7889],
        [55.0856, 51.9060, 48.6219, 45.5683, 42.6152, 39.6836, 36.7812, 33.7308,
         30.9662, 28.7889]], grad_fn=<ViewBackward>)
fm variance: tensor([[0.2526, 0.2130, 0.2023, 0.2000, 0.1998, 0.1997, 0.2001, 0.2027, 0.2162,
         0.3276],
        [0.2526, 0.2130, 0.2023, 0.2000, 0.1998, 0.1997, 0.2001, 0.2027, 0.2162,
         0.3276],
        [0.2526, 0.2130, 0.2023, 0.2000, 0.1998, 0.